<a href="https://colab.research.google.com/github/withlionbuddha/learning.ai/blob/ground/PseudoLabeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 😄 의사 레이블(Pseudo-label)

---
의사 레이블링은 일반적으로 레이블이 일부만 주어졌을 때, 나머지 데이터를 예측하여 성능을 개선하는데 사용됩니다



In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist

In [ ]:
import os
import shutil
# MNIST 데이터셋이 저장된 경로
cache_dir = os.path.expanduser('~/.keras/datasets/mnist.npz')

# 캐시 파일 삭제
if os.path.exists(cache_dir):
    os.remove(cache_dir)
    print("Cached MNIST data deleted.")

Cached MNIST data deleted.


📚 데이터셋 불러오기

---

* mnist은 손으로 쓴 숫자(0-9) 이미지를 모아둔 데이터셋
* train_input은 입력용 훈련이미지데이터
* train_label은 train_input의 정답 데이터
* test_input은 입력용 시험이미지데이터
* test_label은 test_input의 정답 데이터
* train_input, test_input은 0~255 사이의 정수 값(픽셀 값)으로 이루어진 배열

In [ ]:
# MNIST 데이터셋 불러오기
(train_input, train_label), (test_input, test_label) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
print(f"train_input shape: {train_input.shape}")
print(f"axis=0 (batch of images): {train_input.shape[0]}")
print(f"axis=1 (height of each image): {train_input.shape[1]}")
print(f"axis=2 (width of each image): {train_input.shape[2]}")
print(f"train_label shape: {train_label.shape}")

print(f"-----------------------------------")
print(f"test_input shape: {test_input.shape}")
print(f"test_label shape: {test_label.shape}")

train_input shape: (60000, 28, 28)
axis=0 (batch of images): 60000
axis=1 (height of each image): 28
axis=2 (width of each image): 28
train_label shape: (60000,)
-----------------------------------
test_input shape: (10000, 28, 28)
test_label shape: (10000,)


📚 데이터 전처리



---



*  이미지 데이터 형태의 일관성을 위해서 channel demension을 추가한다.
*  입력값(train_expand_input, test_input)은 astype("float32")를 사용하여 데이터를 32비트 부동 소수점 형식으로 변환합니다. 이는 신경망 모델이 실수 값을 처리하는 데 더 적합하기 때문입니다.
* 입력(train_expand_input, test_input) 데이터의 각 이미지의 픽셀 값(0~255)을 255.0으로 나눠서 0과 1 사이의 값으로 스케일링합니다.




In [ ]:
# 데이터 전처리
train_expand_input = np.expand_dims(train_input, axis=-1)
test_expand_input = np.expand_dims(test_input, axis=-1)
print(f"train_expand_input shape: {train_expand_input.shape}")

train_expand_input = train_expand_input.astype("float32") / 255.0
test_expand_input = test_expand_input.astype("float32") / 255.0

train_expand_input shape: (60000, 28, 28, 1)


📚 데이터 타입 확인



---


* numpy.ndarray 는 NumPy 라이브러리의 다차원 배열 객체로, 효율적인 수치 연산

In [ ]:
type(train_expand_input)

numpy.ndarray

📚 데이터 분할 , 데이터 슬라이싱



---


*   train_expand_input[:num_labeled]은 입력용 훈련데이터(train_expand_input)의 6000개 images 중에서 1000개의 images를 분할하여 input_labeled에 할당합니다.



In [ ]:
# 레이블이 있는 데이터를 일부만 사용하고 나머지는 의사 레이블로 처리
# 여기서는 예시로 train_label의 앞부분만 사용
num_labeled = 1000
train_input_labeled = train_expand_input[:num_labeled]
train_label_labeled = train_label[:num_labeled]

input_labeled의 형태는

In [ ]:
print(f"train_input_labeled shape: {train_input_labeled.shape}")
print(f"train_label_labeled shape: {train_label_labeled.shape}")

train_input_labeled shape: (1000, 28, 28, 1)
train_label_labeled shape: (1000,)


In [ ]:
test_input_unlabeled = test_expand_input[num_labeled:]
test_label_unlabeled = test_label[num_labeled:]  # test_label_unlabed는 실제 학습에 사용되지 않음

print(f"test_input_unlabeled shape: {test_input_unlabeled.shape}")
print(f"test_label_unlabeled shape: {test_label_unlabeled.shape}")

test_input_unlabeled shape: (9000, 28, 28, 1)
test_label_unlabeled shape: (9000,)


In [ ]:
# 간단한 모델 구성
model = models.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# 먼저 라벨이 있는 데이터로 학습
model.fit(train_input_labeled , train_label_labeled , epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.3713 - loss: 1.9615 - val_accuracy: 0.7600 - val_loss: 1.0942
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8296 - loss: 0.8061 - val_accuracy: 0.8250 - val_loss: 0.6895
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8991 - loss: 0.4324 - val_accuracy: 0.8400 - val_loss: 0.5937
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9359 - loss: 0.3297 - val_accuracy: 0.8600 - val_loss: 0.5289
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9387 - loss: 0.2689 - val_accuracy: 0.8700 - val_loss: 0.5139


📚 의사 레이블 생성

의사 레이블은 예측된 결과로 새로 생성된 분류의 인덱스 입니다.

---

*   predict()함수에서 입력용 시험데이터(test_input_unlabeled)에 대해서  각 숫자(0~9) 별로 예측된 확율값을 결과값으로 반환합니다.

*   np.argmax() 함수에서 예측된 확율결과값 중에서 가장 높은 확율값의 인덱스를 반환합니다.









In [ ]:
predicted = model.predict(test_input_unlabeled)

282/282 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


array([[2.91681266e-04, 1.00427540e-02, 4.27812862e-04, ...,
        4.01412189e-01, 1.11776209e-02, 5.11810422e-01],
       [9.42501187e-01, 2.59180651e-05, 3.66872083e-03, ...,
        1.02001720e-03, 3.78395664e-03, 9.20158578e-04],
       [2.59616529e-04, 6.06800541e-02, 8.44698250e-01, ...,
        9.14879492e-05, 9.10250098e-02, 5.02939765e-05],
       ...,
       [7.65093719e-05, 4.21224744e-04, 2.00798575e-04, ...,
        3.51952016e-02, 7.12935301e-03, 4.45011497e-01],
       [3.24946009e-02, 5.64597221e-03, 3.01357219e-03, ...,
        1.18357996e-02, 4.58157539e-01, 1.92918796e-02],
       [2.94731377e-04, 1.10422327e-07, 4.40380682e-04, ...,
        1.12150104e-07, 3.59914424e-07, 2.49373898e-07]], dtype=float32)

In [ ]:
print(f"predicted.shape : {predicted.shape}" )
print(f"len(predicted) : {len(predicted)}")
print(f"predicted[0] : {predicted[0]}")
print(f"predicted[1] : {predicted[1]}")
print(f"predicted[2] : {predicted[2]}")

predicted.shape : (9000, 10)
len(predicted) : 9000
predicted[0] : [2.9168127e-04 1.0042754e-02 4.2781286e-04 4.4641491e-02 1.1808919e-02
 7.9182498e-03 4.6881268e-04 4.0141219e-01 1.1177621e-02 5.1181042e-01]
predicted[1] : [9.4250119e-01 2.5918065e-05 3.6687208e-03 2.8646318e-04 8.9032709e-04
 1.6370960e-02 3.0532306e-02 1.0200172e-03 3.7839566e-03 9.2015858e-04]
predicted[2] : [2.5961653e-04 6.0680054e-02 8.4469825e-01 1.7353533e-03 8.1516780e-05
 3.7128458e-04 1.0071534e-03 9.1487949e-05 9.1025010e-02 5.0293977e-05]


In [ ]:
# 의사 레이블 생성: 레이블이 없는 데이터를 모델을 통해 예측
pseudo_labels = np.argmax(predicted, axis=1)
pseudo_labels

array([9, 0, 2, ..., 4, 8, 6])

In [ ]:
len(pseudo_labels)

9000

In [ ]:
# 라벨이 없는 데이터에 대해 예측된 의사 레이블로 다시 학습
x_combined = np.concatenate([train_input_labeled, test_input_unlabeled ])
y_combined = np.concatenate([train_label_labeled, pseudo_labels])

In [ ]:
model.fit(x_combined, y_combined, epochs=5, batch_size=32, validation_split=0.2)

# 테스트 데이터로 성능 확인
test_loss, test_acc = model.evaluate(train_input_labeled, train_label_labeled)
print(f"Test accuracy: {test_acc:.4f}")